In [1]:
using GrootSim

using CSV
using DataFrames
using Dates
using Distributions
using DotEnv
DotEnv.load!()
using PyPlot
using Serialization

using PyCall
pickle = pyimport("pickle");

[ Info: Precompiling GrootSim [84197a3c-ab72-42ed-a031-1046bb28d0ec] (cache misses: include_dependency fsize change (2))


## Evaluation data params

In [2]:
basepath = ENV["EVALUATION_RESULTS_PATH"]
outpath = ENV["PAPER_RESULTS_PATH"];

In [3]:
time_window = "day"
net_models = ["hypergraph", "clique", "graph"]
# involvements = ["high_involvement", "medium_involvement", "low_involvement"]
involvements = ["medium_involvement"]
# thrs = [2, 3, 4]
thrs = [4];

In [4]:
eval_res_path = joinpath(basepath, "$(time_window)");

In [5]:
subreddits_to_eval = [
    "AskReddit",
    "bayarea",
    "California",
    "collapse",
    "news",
    "politics",
    "AskReddit_collapse",
    "AskReddit_news",
    "AskReddit_politics",
    "bayarea_news",
    "collapse_news",
    "collapse_politics",
    "AskReddit_news_politics",
    "collapse_news_politics",
    "all"
];

In [9]:
subreddit_labels = [
    "r/AskReddit",
    "r/bayarea",
    "r/California",
    "r/collapse",
    "r/news",
    "r/politics",
    "r/AskReddit;r/collapse",
    "r/AskReddit;r/news",
    "r/AskReddit;r/politics",
    "r/bayarea;r/news",
    "r/collapse;r/news",
    "r/collapse;r/politics",
    "r/AskReddit;r/news;r/politics",
    "r/collapse;r/news;r/politics",
    "All subreddits"
];

## Across nets

### Criterion C2: average number of intervals by which the model anticipates or postpones the identification of a user's first opinion drift

In [6]:
hg_data = nothing
cl_data = nothing
gr_data = nothing

# loading evaluation data
thrs = 4
eval_func = "metrics"
involvement = "medium_involvement"

data = _loading_data(thrs, subreddits_to_eval, eval_func, eval_res_path)

subreddits = []
class = [
    "AskReddit",
    "bayarea",
    "California",
    "collapse",
    "news",
    "politics",
    "AskRedditCollapse",
    "AskRedditNews",
    "AskRedditPolitics",
    "bayareaNews",
    "collapseNews",
    "collapsePolitics",
    "AskRedditNewsPolitics",
    "collapseNewsPolitics",
    "all"
]

# data to store 

# ANTICIPATED
avg_anticipated_intervals = []
median_anticipated_intervals = []
std_anticipated_intervals = []
#
avg_anticipated_intervals_cl = []
median_anticipated_intervals_cl = []
std_anticipated_intervals_cl = []
#
avg_anticipated_intervals_gr = []
median_anticipated_intervals_gr = []
std_anticipated_intervals_gr = []

# POSTPONED
avg_postponed_intervals = []
median_postponed_intervals = []
std_postponed_intervals = []
#
avg_postponed_intervals_cl = []
median_postponed_intervals_cl = []
std_postponed_intervals_cl = []
#
avg_postponed_intervals_gr = []
median_postponed_intervals_gr = []
std_postponed_intervals_gr = []

# MISMATCHED
avg_mismatched_intervals = []
median_mismatched_intervals = []
std_mismatched_intervals = []
#
avg_mismatched_intervals_cl = []
median_mismatched_intervals_cl = []
std_mismatched_intervals_cl = []
#
avg_mismatched_intervals_gr = []
median_mismatched_intervals_gr = []
std_mismatched_intervals_gr = []


for (idx, entry) in enumerate(data)
    k = entry.first
    d = entry.second
    println(idx, "-", k)

    push!(subreddits, k)

    # series
    # [avg_anticipated_intervals, avg_postponed_intervals, avg_mismatched_intervals]
    hg_data = get_first_drit_data_direction(d, involvement, "hypergraph")
    cl_data = get_first_drit_data_direction(d, involvement, "clique")
    gr_data = get_first_drit_data_direction(d, involvement, "graph")

    # println(hg_data)

    # println(length(hg_data))
    # println(hg_data)

    # ANTICIPATED
    push!(avg_anticipated_intervals, mean(hg_data[1]))
    push!(median_anticipated_intervals, median(hg_data[1]))
    push!(std_anticipated_intervals, std(hg_data[1]))

    push!(avg_anticipated_intervals_cl, mean(cl_data[1]))
    push!(median_anticipated_intervals_cl, median(cl_data[1]))
    push!(std_anticipated_intervals_cl, std(cl_data[1]))

    push!(avg_anticipated_intervals_gr, mean(gr_data[1]))
    push!(median_anticipated_intervals_gr, median(gr_data[1]))
    push!(std_anticipated_intervals_gr, std(gr_data[1]))

    # POSTPONED
    push!(avg_postponed_intervals, mean(hg_data[2]))
    push!(median_postponed_intervals, median(hg_data[2]))
    push!(std_postponed_intervals, std(hg_data[2]))

    push!(avg_postponed_intervals_cl, mean(cl_data[2]))
    push!(median_postponed_intervals_cl, median(cl_data[2]))
    push!(std_postponed_intervals_cl, std(cl_data[2]))

    push!(avg_postponed_intervals_gr, mean(gr_data[2]))
    push!(median_postponed_intervals_gr, median(gr_data[2]))
    push!(std_postponed_intervals_gr, std(gr_data[2]))

    # MISMATCHED
    push!(avg_mismatched_intervals, mean(hg_data[3]))
    push!(median_mismatched_intervals, median(hg_data[3]))
    push!(std_mismatched_intervals, std(hg_data[3]))

    push!(avg_mismatched_intervals_cl, mean(cl_data[3]))
    push!(median_mismatched_intervals_cl, median(cl_data[3]))
    push!(std_mismatched_intervals_cl, std(cl_data[3]))

    push!(avg_mismatched_intervals_gr, mean(gr_data[3]))
    push!(median_mismatched_intervals_gr, median(gr_data[3]))
    push!(std_mismatched_intervals_gr, std(gr_data[3]))

end # for over data

1-AskReddit
2-bayarea
3-California
4-collapse
5-news
6-politics
7-AskReddit_collapse
8-AskReddit_news
9-AskReddit_politics
10-bayarea_news
11-collapse_news
12-collapse_politics
13-AskReddit_news_politics
14-collapse_news_politics
15-all


In [ ]:
# # save csv 
# CSV.write(joinpath(outpath, "Criterion2_direction.csv"), DataFrame(
#     x = collect(1:length(subreddits_to_eval)),
#     class = class,
#     subreddit = subreddits,
#     label = subreddit_labels,
#     # ANTICIPATED
#     avg_anticipated_intervals = avg_anticipated_intervals,
#     median_anticipated_intervals = median_anticipated_intervals,
#     std_anticipated_intervals = std_anticipated_intervals,
#     avg_anticipated_intervals_cl = avg_anticipated_intervals_cl,
#     median_anticipated_intervals_cl = median_anticipated_intervals_cl,
#     std_anticipated_intervals_cl = std_anticipated_intervals_cl,
#     avg_anticipated_intervals_gr = avg_anticipated_intervals_gr,
#     median_anticipated_intervals_gr = median_anticipated_intervals_gr,
#     std_anticipated_intervals_gr = std_anticipated_intervals_gr,
#     # POSTPONED
#     avg_postponed_intervals = avg_postponed_intervals,
#     median_postponed_intervals = median_postponed_intervals,
#     std_postponed_intervals = std_postponed_intervals,
#     avg_postponed_intervals_cl = avg_postponed_intervals_cl,
#     median_postponed_intervals_cl = median_postponed_intervals_cl,
#     std_postponed_intervals_cl = std_postponed_intervals_cl,
#     avg_postponed_intervals_gr = avg_postponed_intervals_gr,
#     median_postponed_intervals_gr = median_postponed_intervals_gr,
#     std_postponed_intervals_gr = std_postponed_intervals_gr,
#     # MISMATCHED
#     avg_mismatched_intervals = avg_mismatched_intervals,
#     median_mismatched_intervals = median_mismatched_intervals,
#     std_mismatched_intervals = std_mismatched_intervals,
#     avg_mismatched_intervals_cl = avg_mismatched_intervals_cl,
#     median_mismatched_intervals_cl = median_mismatched_intervals_cl,
#     std_mismatched_intervals_cl = std_mismatched_intervals_cl,
#     avg_mismatched_intervals_gr = avg_mismatched_intervals_gr,
#     median_mismatched_intervals_gr = median_mismatched_intervals_gr,
#     std_mismatched_intervals_gr = std_mismatched_intervals_gr
# ))